In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
data = pickle.load(open("herd.pickle", 'rb'))

ear_pos_mean = list(torch.mean(torch.stack(list(zip(*list(zip(*data))[1]))[i], dim=0), dim=0) for i in range(2))
ear_pos_stdvar = list(torch.sqrt(torch.mean(torch.var(torch.stack(list(zip(*list(zip(*data))[1]))[i], dim=0), dim=0))) for i in range(2))

right_ear_tip_pos = data[0][1][0]
left_ear_tip_pos = data[0][1][1]
right_ear_tip_dir = right_ear_tip_pos/torch.norm(right_ear_tip_pos)
left_ear_tip_dir = left_ear_tip_pos/torch.norm(left_ear_tip_pos)

# We need a better template than that
template = data[0][0]
template = dm.Utilities.gaussian_kernel_smooth(template, 0.1)
herd = list(list(zip(*data))[0])[1:-1]

print(template.shape)
print(len(herd))

ear_sigma = 0.6

torch.Size([107, 2])
14


In [3]:
%matplotlib qt5
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=4.)
for bunny in herd:
    plt.plot(bunny.numpy()[:, 0], bunny.numpy()[:, 1], '-', color='C4')
# plt.plot(ear_pos_mean[0][0].numpy(), ear_pos_mean[0][1].numpy(), 'X')
# plt.plot(ear_pos_mean[1][0].numpy(), ear_pos_mean[1][1].numpy(), 'X')
plt.quiver(right_ear_tip_pos[0].numpy(), right_ear_tip_pos[1].numpy(), right_ear_tip_dir[0].numpy(), right_ear_tip_dir[1].numpy(), color='red')
plt.quiver(left_ear_tip_pos[0], left_ear_tip_pos[1], left_ear_tip_dir[0], left_ear_tip_dir[1], color='blue')
plt.axis('equal')
plt.show()


In [4]:
sigma_rotation = 4.
coeff_rotation = 0.005
coeff_translation = 0.5

right_ear_translation = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=(right_ear_tip_pos.clone().unsqueeze(0).requires_grad_(), right_ear_tip_dir.clone().unsqueeze(0).requires_grad_()), label='right_ear_translation')
left_ear_translation = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=(left_ear_tip_pos.clone().unsqueeze(0).requires_grad_(), left_ear_tip_dir.clone().unsqueeze(0).requires_grad_()), label='left_ear_translation')
#rigid_rotation = dm.DeformationModules.LinearDeformation.build(torch.tensor([[0., -1.], [1., 0.]]), gd=torch.tensor([[0., 0.]], requires_grad=True))
rigid_translation = dm.DeformationModules.GlobalTranslation(2, coeff_translation)
local_rotation = dm.DeformationModules.LocalRotation(2, sigma_rotation, coeff=coeff_rotation, gd=torch.tensor([[0., 0.]], requires_grad=True))

targets = herd

atlas = dm.Models.Atlas(template, [right_ear_translation, left_ear_translation, local_rotation, rigid_translation], [dm.Attachment.VarifoldAttachment(2, [0.3, 1.])], len(targets), fit_gd=None, lam=100.)

In [ ]:
fitter = dm.Models.ModelFittingScipy(atlas, 1.)
costs = fitter.fit(targets, 3000, options={'shoot_method': 'rk4'})

Initial energy = 54922.344
Iteration: 1 
Total energy = 50368.226562 
Attach cost = 50367.820312 
Deformation cost = 0.406464
Iteration: 2 
Total energy = 48056.859375 
Attach cost = 48051.246094 
Deformation cost = 5.613342
Iteration: 3 
Total energy = 40174.871094 
Attach cost = 40130.699219 
Deformation cost = 44.174484
Iteration: 4 
Total energy = 35535.484375 
Attach cost = 35463.296875 
Deformation cost = 72.185493
Iteration: 5 
Total energy = 32838.804688 
Attach cost = 32769.363281 
Deformation cost = 69.442833
Iteration: 6 
Total energy = 31730.576172 
Attach cost = 31667.824219 
Deformation cost = 62.752781
Iteration: 7 
Total energy = 29782.144531 
Attach cost = 29721.050781 
Deformation cost = 61.096024
Iteration: 8 
Total energy = 28845.925781 
Attach cost = 28786.900391 
Deformation cost = 59.024494
Iteration: 9 
Total energy = 27441.234375 
Attach cost = 27374.697266 
Deformation cost = 66.534744
Iteration: 10 
Total energy = 26278.396484 
Attach cost = 26203.058594 
Def

Iteration: 46 
Total energy = 17123.896484 
Attach cost = 17057.373047 
Deformation cost = 66.524765
Iteration: 47 
Total energy = 17054.001953 
Attach cost = 16987.207031 
Deformation cost = 66.795891
Iteration: 48 
Total energy = 17020.953125 
Attach cost = 16953.458984 
Deformation cost = 67.495155
Iteration: 49 
Total energy = 16906.773438 
Attach cost = 16839.404297 
Deformation cost = 67.370148
Iteration: 50 
Total energy = 16867.595703 
Attach cost = 16800.312500 
Deformation cost = 67.281075
Iteration: 51 
Total energy = 16821.869141 
Attach cost = 16754.609375 
Deformation cost = 67.261612
Iteration: 52 
Total energy = 16755.191406 
Attach cost = 16688.093750 
Deformation cost = 67.095894
Iteration: 53 
Total energy = 16650.412109 
Attach cost = 16583.500000 
Deformation cost = 66.914948
Iteration: 54 
Total energy = 16443.535156 
Attach cost = 16377.061523 
Deformation cost = 66.472977
Iteration: 55 
Total energy = 16261.002930 
Attach cost = 16195.007812 
Deformation cost = 

Iteration: 92 
Total energy = 14610.863281 
Attach cost = 14541.187500 
Deformation cost = 69.676292
Iteration: 93 
Total energy = 14608.001953 
Attach cost = 14538.261719 
Deformation cost = 69.740868
Iteration: 94 
Total energy = 14605.837891 
Attach cost = 14536.076172 
Deformation cost = 69.763222
Iteration: 95 
Total energy = 14603.335938 
Attach cost = 14533.573242 
Deformation cost = 69.762962
Iteration: 96 
Total energy = 14599.779297 
Attach cost = 14530.041016 
Deformation cost = 69.738220
Iteration: 97 
Total energy = 14596.850586 
Attach cost = 14527.155273 
Deformation cost = 69.695549
Iteration: 98 
Total energy = 14592.525391 
Attach cost = 14522.851562 
Deformation cost = 69.674477
Iteration: 99 
Total energy = 14589.110352 
Attach cost = 14519.448242 
Deformation cost = 69.662445
Iteration: 100 
Total energy = 14586.444336 
Attach cost = 14516.789062 
Deformation cost = 69.653732
Iteration: 101 
Total energy = 14583.408203 
Attach cost = 14513.655273 
Deformation cost 

Iteration: 137 
Total energy = 14347.588867 
Attach cost = 14277.970703 
Deformation cost = 69.617416
Iteration: 138 
Total energy = 14346.318359 
Attach cost = 14276.686523 
Deformation cost = 69.630234
Iteration: 139 
Total energy = 14345.732422 
Attach cost = 14276.113281 
Deformation cost = 69.618500
Iteration: 140 
Total energy = 14344.351562 
Attach cost = 14274.732422 
Deformation cost = 69.618584
Iteration: 141 
Total energy = 14343.429688 
Attach cost = 14273.826172 
Deformation cost = 69.602692
Iteration: 142 
Total energy = 14342.563477 
Attach cost = 14272.988281 
Deformation cost = 69.575607
Iteration: 143 
Total energy = 14341.629883 
Attach cost = 14272.083008 
Deformation cost = 69.545609
Iteration: 144 
Total energy = 14341.415039 
Attach cost = 14271.915039 
Deformation cost = 69.499153
Iteration: 145 
Total energy = 14340.087891 
Attach cost = 14270.580078 
Deformation cost = 69.507637
Iteration: 146 
Total energy = 14339.806641 
Attach cost = 14270.287109 
Deformati

Iteration: 182 
Total energy = 14265.516602 
Attach cost = 14195.542969 
Deformation cost = 69.973450
Iteration: 183 
Total energy = 14264.862305 
Attach cost = 14194.909180 
Deformation cost = 69.951096
Iteration: 184 
Total energy = 14264.366211 
Attach cost = 14194.411133 
Deformation cost = 69.956131
Iteration: 185 
Total energy = 14263.361328 
Attach cost = 14193.396484 
Deformation cost = 69.964806
Iteration: 186 
Total energy = 14262.944336 
Attach cost = 14192.993164 
Deformation cost = 69.953545
Iteration: 187 
Total energy = 14262.567383 
Attach cost = 14192.622070 
Deformation cost = 69.942902
Iteration: 188 
Total energy = 14262.421875 
Attach cost = 14192.492188 
Deformation cost = 69.932243
Iteration: 189 
Total energy = 14262.312500 
Attach cost = 14192.384766 
Deformation cost = 69.925705
Iteration: 190 
Total energy = 14261.937500 
Attach cost = 14192.028320 
Deformation cost = 69.908440
Iteration: 191 
Total energy = 14261.381836 
Attach cost = 14191.494141 
Deformati

...

Iteration: 902 
Total energy = 13150.400391 
Attach cost = 13072.705078 
Deformation cost = 77.694931
Iteration: 903 
Total energy = 13149.685547 
Attach cost = 13072.000977 
Deformation cost = 77.683754
Iteration: 904 
Total energy = 13148.775391 
Attach cost = 13071.089844 
Deformation cost = 77.685440
Iteration: 905 
Total energy = 13146.902344 
Attach cost = 13069.195312 
Deformation cost = 77.707672
Iteration: 906 
Total energy = 13144.560547 
Attach cost = 13066.805664 
Deformation cost = 77.756416
Iteration: 907 
Total energy = 13142.143555 
Attach cost = 13064.304688 
Deformation cost = 77.838936
Iteration: 908 
Total energy = 13141.244141 
Attach cost = 13063.369141 
Deformation cost = 77.875534
Iteration: 909 
Total energy = 13140.425781 
Attach cost = 13062.516602 
Deformation cost = 77.909813
Iteration: 910 
Total energy = 13138.914062 
Attach cost = 13060.951172 
Deformation cost = 77.962914
Iteration: 911 
Total energy = 13136.048828 
Attach cost = 13057.966797 
Deformati

Iteration: 947 
Total energy = 13032.203125 
Attach cost = 12949.625000 
Deformation cost = 82.578453
Iteration: 948 
Total energy = 13030.965820 
Attach cost = 12948.322266 
Deformation cost = 82.641891
Iteration: 949 
Total energy = 13030.572266 
Attach cost = 12947.884766 
Deformation cost = 82.687965
Iteration: 950 
Total energy = 13027.409180 
Attach cost = 12944.648438 
Deformation cost = 82.761787
Iteration: 951 
Total energy = 13026.242188 
Attach cost = 12943.467773 
Deformation cost = 82.773643
Iteration: 952 
Total energy = 13024.408203 
Attach cost = 12941.571289 
Deformation cost = 82.837166
Iteration: 953 
Total energy = 13023.733398 
Attach cost = 12940.959961 
Deformation cost = 82.771164
Iteration: 954 
Total energy = 13022.356445 
Attach cost = 12939.568359 
Deformation cost = 82.787872
Iteration: 955 
Total energy = 13021.655273 
Attach cost = 12938.859375 
Deformation cost = 82.795128
Iteration: 956 
Total energy = 13020.412109 
Attach cost = 12937.625977 
Deformati

Iteration: 992 
Total energy = 12873.243164 
Attach cost = 12786.282227 
Deformation cost = 86.962532
Iteration: 993 
Total energy = 12868.335938 
Attach cost = 12781.092773 
Deformation cost = 87.243080
Iteration: 994 
Total energy = 12866.226562 
Attach cost = 12779.177734 
Deformation cost = 87.047981
Iteration: 995 
Total energy = 12864.688477 
Attach cost = 12777.537109 
Deformation cost = 87.152618
Iteration: 996 
Total energy = 12863.741211 
Attach cost = 12776.541992 
Deformation cost = 87.198982
Iteration: 997 
Total energy = 12862.388672 
Attach cost = 12775.197266 
Deformation cost = 87.190369
Iteration: 998 
Total energy = 12860.630859 
Attach cost = 12773.463867 
Deformation cost = 87.166801
Iteration: 999 
Total energy = 12859.791016 
Attach cost = 12772.163086 
Deformation cost = 87.627373
Iteration: 1000 
Total energy = 12856.802734 
Attach cost = 12769.496094 
Deformation cost = 87.307022
Iteration: 1001 
Total energy = 12855.822266 
Attach cost = 12768.437500 
Deforma

Iteration: 1037 
Total energy = 12800.118164 
Attach cost = 12709.485352 
Deformation cost = 90.632782
Iteration: 1038 
Total energy = 12797.125000 
Attach cost = 12706.307617 
Deformation cost = 90.817947
Iteration: 1039 
Total energy = 12795.302734 
Attach cost = 12704.474609 
Deformation cost = 90.827065
Iteration: 1040 
Total energy = 12793.147461 
Attach cost = 12702.348633 
Deformation cost = 90.799980
Iteration: 1041 
Total energy = 12789.269531 
Attach cost = 12698.419922 
Deformation cost = 90.849937
Iteration: 1042 
Total energy = 12781.935547 
Attach cost = 12690.954102 
Deformation cost = 90.981583
Iteration: 1043 
Total energy = 12775.735352 
Attach cost = 12684.356445 
Deformation cost = 91.377960
Iteration: 1044 
Total energy = 12773.608398 
Attach cost = 12682.049805 
Deformation cost = 91.557320
Iteration: 1045 
Total energy = 12767.287109 
Attach cost = 12675.243164 
Deformation cost = 92.044403
Iteration: 1046 
Total energy = 12765.744141 
Attach cost = 12673.950195 

Iteration: 1082 
Total energy = 12700.525391 
Attach cost = 12602.503906 
Deformation cost = 98.020172
Iteration: 1083 
Total energy = 12699.488281 
Attach cost = 12601.333008 
Deformation cost = 98.155518
Iteration: 1084 
Total energy = 12699.043945 
Attach cost = 12600.744141 
Deformation cost = 98.298309
Iteration: 1085 
Total energy = 12697.756836 
Attach cost = 12599.287109 
Deformation cost = 98.469902


In [ ]:
%matplotlib qt5
plt.plot(range(len(costs)), costs)
plt.show()

In [8]:
print(atlas.models[0].init_manifold[1].gd)
print(atlas.models[0].init_manifold[2].gd)
print(atlas.models[0].init_manifold[3].gd)

(tensor([[0.6164, 1.4765]], requires_grad=True), tensor([[0.3852, 0.9228]], requires_grad=True))
(tensor([[-1.0290,  1.2253]], requires_grad=True), tensor([[-0.6431,  0.7658]], requires_grad=True))
tensor([[0., 0.]], requires_grad=True)


In [9]:
%matplotlib qt5
shoot_it = 10
shoot_method = 'rk4'
it_per_snapshot = 2
snapshots = int(shoot_it/it_per_snapshot)

disp_targets = len(targets)

for i in range(disp_targets):
    silent_pos = atlas.models[i].init_manifold[0].gd.detach().clone()
    silent_mom = atlas.models[i].init_manifold[0].cotan.detach().clone()
    right_ear_trans_gd = (atlas.models[i].init_manifold[1].gd[0].detach().clone(),
                          atlas.models[i].init_manifold[1].gd[1].detach().clone())
    right_ear_trans_mom = (atlas.models[i].init_manifold[1].cotan[0].detach().clone(),
                           atlas.models[i].init_manifold[1].cotan[1].detach().clone())
    left_ear_trans_gd = (atlas.models[i].init_manifold[2].gd[0].detach().clone(),
                          atlas.models[i].init_manifold[2].gd[1].detach().clone())
    left_ear_trans_mom = (atlas.models[i].init_manifold[2].cotan[0].detach().clone(),
                          atlas.models[i].init_manifold[2].cotan[1].detach().clone())
    local_rotation_gd = atlas.models[i].init_manifold[3].gd.detach().clone()
    local_rotation_mom = atlas.models[i].init_manifold[3].cotan.detach().clone()

    silent = dm.DeformationModules.SilentLandmarks(2, silent_pos.shape[0], gd=silent_pos, cotan=silent_mom)
    right_ear_trans = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=right_ear_trans_gd, cotan=right_ear_trans_mom)
    left_ear_trans = dm.DeformationModules.OrientedTranslations(2, 1, ear_sigma, gd=left_ear_trans_gd, cotan=left_ear_trans_mom)
    #global_trans = dm.DeformationModules.GlobalTranslation(2)
    local_rot = dm.DeformationModules.LocalRotation(2, sigma_rotation, coeff=coeff_rotation, gd=local_rotation_gd, cotan=local_rotation_mom)
    rigid_translation = dm.DeformationModules.GlobalTranslation(2)

    h = dm.HamiltonianDynamic.Hamiltonian([silent, right_ear_trans, left_ear_trans, local_rot, rigid_translation])
    intermediate_states, _ = dm.HamiltonianDynamic.shoot(h, shoot_it, shoot_method, intermediates=True)

    for j in range(snapshots):
        pos = intermediate_states[it_per_snapshot*j].gd[0].numpy()
        pos_right_ear_trans = intermediate_states[it_per_snapshot*j].gd[1][0].numpy()
        pos_left_ear_trans = intermediate_states[it_per_snapshot*j].gd[2][0].numpy()

        plt.subplot(disp_targets, snapshots + 1, i*snapshots + j + i + 1)
        plt.plot(pos[:, 0], pos[:, 1])
        plt.plot(pos_right_ear_trans[:, 0], pos_right_ear_trans[:, 1], 'x')
        plt.plot(pos_left_ear_trans[:, 0], pos_left_ear_trans[:, 1], 'x')
        plt.plot(targets[i].numpy()[:, 0], targets[i].numpy()[:, 1])
        plt.axis('equal')


    final_pos = intermediate_states[-1].gd[0].numpy()
    plt.subplot(disp_targets, snapshots + 1, i*snapshots + snapshots + i + 1)
    plt.plot(targets[i].numpy()[:, 0], targets[i].numpy()[:, 1])
    plt.plot(final_pos[:, 0], final_pos[:, 1])
    plt.axis('equal')

plt.show()